Импортируем библиотеки

In [1]:
import requests as req
import bs4  as bs
from collections import Counter as c
import pandas as pd
import sqlite3

Определяем константы и перемнные

In [2]:
URL_SEARCH = "https://hh.ru/search/vacancy?text=python+middle"
URL_HEADERS = {'User-Agent': 'Mozilla/5.0'}
DB_NAME = 'pars.db'
TABLE_NAME = 'vacancies'

In [3]:
numersOfPage = 0
listLinkOfVacancy = []
vacancyDetailInfo =[]
url_params = {'page':0}

Определяем функции

In [4]:
# Загружаем страницу и возвращаем BeautifulSoup
def getBSPage(URL_SEARCH):
    result = req.get(URL_SEARCH, headers=URL_HEADERS, params=url_params)
    return bs.BeautifulSoup(result.content,'lxml')

In [5]:
# Функция для парсинга страницы и записи ссылок на вакансии
def addVacancyToList(soup):
    for vac in list(soup.find_all('a',{'data-qa': 'serp-item__title','target':'_blank'})):
        vac = str(vac)
        if vac.find('hh.ru/vacancy/') > 0 :
            listLinkOfVacancy.append('https://' + vac[vac.find('hh.ru/vacancy/'):vac.find('?')])

Собираем информацию о всех вакансиях

In [6]:
# Получаем кол-во страниц с результатами поиска, собираем список вакансий
soup = getBSPage(URL_SEARCH)
numers_of_page = int(soup.find_all('a',{'data-qa': 'pager-page'})[-1].text)
for i in range(numers_of_page + 1):
    if i == 0:
        addVacancyToList(soup)
    else:
        url_params['page'] = i
        soup = getBSPage(URL_SEARCH)
        addVacancyToList(soup)

Собираем информацию по всем вакансиям из созданного ранее списка

In [7]:
for link in listLinkOfVacancy:
    vacancyRow = []
    listOfRequirements = []
    soup = getBSPage(link)
    companyName = soup.find('a',{'data-qa':'vacancy-company-name','class':'bloko-link bloko-link_kind-tertiary'})
    if (companyName is not None):
        companyName = companyName.text
    vacancyName = soup.find('h1', {'class':'bloko-header-section-1','data-qa':'vacancy-title'})
    if (vacancyName is not None):
        vacancyName = vacancyName.text
    vacancyDescription = ''
    for row in soup.find_all('div',{'class':'vacancy-section'}):
        description = row.find('div',{'class':'g-user-content','data-qa':'vacancy-description'})
        if (description is not None):
            vacancyDescription=description
    vacancies = soup.find_all('div', {'class': 'vacancy-section'})
    for vacancy in vacancies:
        if vacancy.find('h2',{'class':'bloko-header-section-2'}):
            requirements = vacancy.find_all('span')
            for requirement in requirements:
                listOfRequirements.append(requirement.text)
    vacancyRow.append(companyName)
    vacancyRow.append(vacancyName)
    vacancyRow.append(vacancyDescription)
    vacancyRow.append(listOfRequirements)
    vacancyDetailInfo.append(vacancyRow)

Выводим результат

In [ ]:
columns = ['company_name', 'position', 'job_description', 'key_skills']
df = pd.DataFrame(data=vacancyDetailInfo,columns=columns)

In [ ]:
conn = sqlite3.connect(DB_NAME)
df.to_sql(TABLE_NAME,conn,if_exists='append',index=False)